# Underlying

# NormalAsset

## _generate_base

Returns the underlying following a standard  Brownian motion. For each simulation and for each valuation date we have $S(t)$. That is to say, for the $i$-th simulation and the $k$-th valuation date we have:

$$
\textrm{base}[i, j, k] = \Delta Y_{i,k}(t^1_{j-1,k}), \quad j\in\{0, \ldots, J+1\},
$$
where 
$$
t^1_{j-1,k}:= \begin{cases} t^1_{j} \qquad \text{ for }j\geq 0 \quad \text{ (independent of } k\text{)}\,,\\
t^0_{k} \qquad \text{ for }j=-1\,,
\end{cases}
$$
which is consistent with the definition of `intervals` given below. The simulation depends on the starting date as, for instance, 
$$
Y_{i,k}(t^1_{0-1,k}) = Y_{i,k}(t^0_{k}) = Y(t_k^0)\,,
$$
which is a common value for all simulations for that valuation date. 

### Technical note

For the sake of educational purposes, let us give a full-detailed description of the actual implementation of this part, with a full vectorization is as follows (no loop needed).Firstly, an adjustment is necessary to account for the expected impact of dividends in the pricing model. 

-If `forward_measure = True`, then, $\mu=-d$ and,

-If `forward_measure = False`, $\mu=\text{{self.drift}}-d$, where the pre-defined drift value, `self.drift`, must also be taken into account.


Let's assume that  $S_t$ follows a standard geometric Brownian motion, i.e.,
$$
dS_t = \mu S_t dt + \sigma S_t dW_t\,.
$$
Let $Y := \log(S)$, so $Y$ follows a normal distribution. Using the Itô rule,
$$
dY_t =  \left(\mu -\frac12 \sigma^2\right) dt + \sigma d W_t \,,
$$
which can be easily integrated between $s$ and $t$. 
$$
Y_t - Y_s = \left(\mu -\frac12 \sigma^2\right)  (t-s) + \sigma Z\sqrt{t-s} \,,
$$
with $Z\sim\mathcal{N}(0,1).$ Then the `base[i,j,k]` of the Normal Asset $Y_t$  is given by
$$
\Delta_j{Y}^{i,k} := {Y}^{i,k}(t^1_{j})-{Y}^{i,k}(t^1_{j-1}) = \left(\mu_k -\frac12 \sigma_k^2\right) \textrm{intervals}_{jk} + \sigma_k \cdot \textrm{draws}_{ijk} \cdot \sqrt{\textrm{intervals}_{jk}}\,,
$$
where 
$$
\textrm{intervals}[j,k]= \begin{cases}\textrm{intervals}[j,k]=\text{DCC}(t^1_{j-1}, t^1_{j})\text{ for }j>0 \text{ (independent of } k\text{)}\,,\\
\textrm{intervals}[0,k]=\text{DCC}(t^0_{k}, t^1_{0})\text{ for }j=0\,.
\end{cases}
$$
$\text{DCC}$ represents the day count convention, see [Brigo & Mercurio, 2006], and will depend on the calendar used. For simplicity, we can think of it as $\text{DCC}(t,t')=t'-t$ as a year fraction. Also, 

- `draws[i,j,k]` is a sample of a random variable for the $i$-th simulation and the $k$-th valuation date for the $j$-th step (simulation date).

 $\text{Code}$ - math $(\TeX)$ **translation**

* `start_dates`      $\rightarrow \{t^0_k\}_{k=0}^K$ ,
* `step_dates`      $\rightarrow \{t^1_j\}_{j=0}^J$ ,
* `draws`    $\rightarrow$     normal draws as above,
* `intervals` $\rightarrow$ intervals as above,
* `forward_measure` $\rightarrow$ use of the forward measure as explained above (`forward_measure = True` by default).

## generate_paths

It returns the simulation paths for the asset price. For the $i$-th simulation and the $k$-th valuation date if we denote by `paths` the output of this method we have
$$
\textrm{paths}[i,j,k] = Y_{i,k}(t^1_{j-1,k}), \quad j\in\{0, \ldots, J+1\},
$$ 
where 
$$
t^1_{j-1,k}:= \begin{cases} t^1_{j} \qquad \text{ for }j\geq 0 \quad \text{ (independent of } k\text{)}\,,\\
t^0_{k} \qquad \text{ for }j=-1\,,
\end{cases}
$$
which is consistent with the definition of `intervals` given above. The simulation depends on the starting date as, for instance, 
$$
Y_{i,k}^l(t^1_{0-1,k}) = Y_{i,k}^l(t^0_{k}) = Y^l(t_k^0)\,,
$$
which is a common value for all simulations for that valuation date, the initial condition that would be obtained using market data. Similarly, parameters can depend on the simulation date.

 $\text{Code}$ - math $(\TeX)$ **translation**

* `start_dates`      $\rightarrow \{t^0_k\}_{k=0}^K$ ,
* `step_dates`      $\rightarrow \{t^1_j\}_{j=0}^J$ ,
* `draws`    $\rightarrow$     normal draws as above,
* `intervals` $\rightarrow$ intervals as above,
* `forward_measure` $\rightarrow$ use of the forward measure as explained above (`forward_measure = True` by default).

# LogNormalAsset

## _generate_prices

Applies the log of the Normal Asset method. For the conventions and details on the prices  see the "Technical note" in the [NormalAsset section](#NormalAsset).

## generate_paths

Applies the exponential of the Normal Asset method. For the conventions and details on the paths  see the "Technical note" in the [NormalAsset section](#NormalAsset).

### Technical note

Let us give a full-detailed description of the actual implementation of this part. In this case, as explained above, $Y_t= \log(S_t)$, then the path of the LogNormal Asset $S_t$ is defined as, 
$$
S_t = e^{Y_t}= e^{Y_0+ \sum_{i=1}^t \Delta Y_i} = e^{\log(S_0)+ \sum_{i=1}^t \Delta Y_i}= S_0 e^{\sum_{i=1}^t \Delta Y_i}
$$
where each $Y_i$ follows a normal distribution. 

# VolModel

Parent class.

## compute_implied_vol

For a given set of option prices, it returns the implied volatility according to the specified Black ('lognormal') or Bachelier's formula ('normal').

# Heston

A class for simulating the Heston stochastic volatility model using Andersen's Quadratic Exponential (QE) method.

The Heston model is a widely used model for the dynamics of asset prices and their volatilities. The QE method is an efficient discretization scheme for
simulating the Heston model, particularly for cases with high mean reversion and low volatility of volatility.

We will follow the notation used in [Andersen, 2008].

## option_price_functions

Return European option prices (Call/Put) using the Fast Fourier Transform (FFT) algorithm based on Lewis expression [Lewis, 2000]. We use the library pyfeng. The output for a Call option is

\begin{equation*}
\textrm{prices}[i,j] = C(\vec{\theta},K_i,T_j)
\end{equation*}
where $C(\vec{\theta},K_i,T_j)$ is the price of the Call option (the same holds for a put option).

 $\text{Code}$ - math $(\TeX)$ **translation**

* `params`      $\rightarrow$  $\vec{\theta} = (\kappa, \epsilon, \theta, \rho, v_0)$ [Andersen, 2008] parameters of the model,
* `strikes`    $\rightarrow$  $\{K_i\}_i$   strikes of the options,
* `times_exp` $\rightarrow$ $\{T_j\}_j$ times to expiry of the options (in years).

## fit_to_options

Adjust parameters to option prices. The objective function is:
$$
F(\vec{\theta}) = \sum_{i=1}^N (C(\vec{\theta}; K_i, T_i)-\tilde{C}(K_i, T_i))^2
$$
where $\tilde{C}(K_i, T_i)$ is the market price of a call option (same for put) at $t$, $N$ is the number of options used for calibration, $\{K_i\}_i$   strikes of the options and $\{T_j\}_j$ times to expiry of the options (in years). **We CAN AlSO divide by the vega due to the reasons explain in page 704 of Vol. III in [Andersen & Piterbarg].**

 $\text{Code}$ - math $(\TeX)$ **translation**

* `date`      $\rightarrow$ valuation date $t$,
* `options_df`      $\rightarrow$ dataframe defining $\tilde{C}(K_i, T_i)$. The columns are the times to expiry (in years) $\{T_j\}_j$ and the index the strikes $\{K_i\}_i$,
* `kind`    $\rightarrow$ $\text{call}$ ($\tilde{C}(K_i, T_i)$) or $\text{put}$ ($\tilde{P}(K_i, T_i)$),
* `recal` $\rightarrow$ if `recal = True` stochastic optimization is used for obtaining an initial guess for `optimize.minimize`.

## generate_paths_vol

Returns the variance in the Heston model using Andersen's Quadratic Exponential (QE) method. For each simulation and for each valuation date we have $V(t)$ as described in [Andersen, 2008]. That is to say, for the $i$-th simulation and the $k$-th valuation date we have:

$$
\textrm{vt}[i, j, k, l=0] = \hat{V}_{i,k}(t^1_{j-1,k}), \quad j\in\{0, \ldots, J+1\},
$$
where 
$$
t^1_{j-1,k}:= \begin{cases} t^1_{j} \qquad \text{ for }j\geq 0 \quad \text{ (independent of } k\text{)}\,,\\
t^0_{k} \qquad \text{ for }j=-1\,,
\end{cases}
$$
which is consistent with the definition of `intervals` given below.

 $\text{Code}$ - math $(\TeX)$ **translation**

* `start_dates`      $\rightarrow \{t^0_k\}_{k=0}^K$ ,
* `draws`    $\rightarrow$     draws for the simulation, $Z_V$ describen in [the next section](##generate_paths),
* `intervals` $\rightarrow$ intervals for the simulation:
$$
\textrm{intervals}[j,k]= \begin{cases}\textrm{intervals}[j,k]=\text{DCC}(t^1_{j-1}, t^1_{j})\text{ for }j>0 \text{ (independent of } k\text{)}\,,\\
\textrm{intervals}[0,k]=\text{DCC}(t^0_{k}, t^1_{0})\text{ for }j=0\,,
\end{cases}
$$

where $\text{DCC}$ represents the day count convention, see Chapter 1 of BM, and will depend on the calendar used. For simplicity, we can think of it as $\text{DCC}(t,t')=t'-t$ as a year fraction.

## generate_paths

It returns the simulation paths for the asset price. Following the notation of [Andersen, 2008], for the $i$-th simulation and the $k$-th valuation date if we denote by `paths` the output of this method we have
$$
\textrm{paths}[i,j,k,l=0] = \hat{X}_{i,k}(t^1_{j-1,k}), \quad j\in\{0, \ldots, J+1\},
$$ 
where 
$$
t^1_{j-1,k}:= \begin{cases} t^1_{j} \qquad \text{ for }j\geq 0 \quad \text{ (independent of } k\text{)}\,,\\
t^0_{k} \qquad \text{ for }j=-1\,,
\end{cases}
$$
which is consistent with the definition of `intervals` given above.

Note that in this case, if **CHANGE**

`draws = self.generate_draws(corr=corr, no_obsdates, no_valuation_dates, no_sims)` (inside `DeterministicVolDiffusionMC`)

with `corr` = $I_2$, $2\times2$ identity matrix, we have:

- `draws[i,j,k,0]` is a sample of a random variable ($Z_V$) for the i-th simulation and the k-th valuation date for the j-th step (simulation date).
- `draws[i,j,k,1]` is a sample of a random variable ($Z$) for the i-th simulation and the k-th valuation date for the j-th step (simulation date).

# MultiAssetHeston(MultiAsset)

## generate_paths

It returns the simulation paths for multiple asset prices using the MultiAssetHeston model. Following the notation of [Andersen, 2008], for the $i$-th simulation and the $k$-th valuation date if we denote by `paths` the output of this method, we have for the $l$-th asset
$$
\textrm{paths}[i,j,k,l] = \hat{X}^l_{i,k}(t^1_{j-1,k}), \quad j\in\{0, \ldots, J+1\},
$$ 
where 
$$
t^1_{j-1,k}:= \begin{cases} t^1_{j} \qquad \text{ for }j\geq 0 \quad \text{ (independent of } k\text{)}\,,\\
t^0_{k} \qquad \text{ for }j=-1\,,
\end{cases}
$$
which is consistent with the definition of `intervals` given above.

In this case, if we have `draws = self.generate_draws(corr=corr, no_obsdates, no_valuation_dates, no_sims)` (inside `DeterministicVolDiffusionMC`), with `corr` being the with `corr` = $I_{2\times (L+1), 2\times (L+1)}$, the identity matrix.

- `draws[i,j,k,l]` is a sample of a random variable ($Z_V^l$) for the $i$-th simulation, the $k$-th valuation date, and the $l$-th asset for the $j$-th step (simulation date).
- `draws[..., self.no_assets:]` corresponds to the vector of i.i.d. standard normal variables ($Z$) used in (A.24) of Mokone. This must be properly transformed so that we can use Andersen's approach (`Heston.generate_paths` here) for a single asset. In particular, from the aforementioned (A.24) and (9.85) of Andersen and Piterbarg, we have:
$$
Z^{\text{Andersen}} = L^* D Z,\qquad D=\text{diag}\left(\frac1{\sqrt{1-\rho_0^2}},\ldots,\frac1{\sqrt{1-\rho_L^2}}\right).
$$

Here, $l$ ranges from $0$ to $L$.


# SABR

A class for simulating the SABR stochastic volatility model using the stochastic Alpha Beta Rho stochastic volatility (SABR-SV) model, which is widely used in the fi-
nancial industry for the pricing of fixed income instruments. We will follow the notation used in [Chen et al., 2011].

## generate_paths_vol

Returns the variance in the SABR model following a standard geometric Brownian motion, p.18 of Chen et al. For each simulation and for each valuation date we have $\sigma(t)$ as described in [Chen et al., 2011]. That is to say, for the $i$-th simulation and the $k$-th valuation date we have:

$$
\textrm{paths}[i, j, k] = \hat{\sigma}_{i,k}(t^1_{j-1,k}), \quad j\in\{0, \ldots, J+1\},
$$
where 
$$
t^1_{j-1,k}:= \begin{cases} t^1_{j} \qquad \text{ for }j\geq 0 \quad \text{ (independent of } k\text{)}\,,\\
t^0_{k} \qquad \text{ for }j=-1\,,
\end{cases}
$$
which is consistent with the definition of `intervals` given below. The simulation depends on the starting date as, for instance, 
$$
\hat{\sigma}_{i,k}(t^1_{0-1,k}) = \hat{\sigma}_{i,k}(t^0_{k}) = \sigma(t_k^0)\,,
$$
which is a common value for all simulations for that valuation date. For observable parameters, the initial condition would be obtained using market data, although here this parameter can be subject to calibration. Similarly, parameters can depend on the simulation date.


### Technical note

For the sake of educational purposes, let us give a full-detailed description of the actual implementation of this part, with a full vectorization is as follows (no loop needed). As $\sigma$ follows a standard geometric Brownian motion with zero drift, i.e.,
$$
d\sigma_t = \alpha \sigma_t dW_t\,,
$$
let $Y := \log(\sigma)$, using the Itô rule,
$$
dY_t = -\frac12 \alpha^2 dt + \alpha d W_t \,,
$$
which can be easily integrated between $s$ and $t$. 
$$
Y_t - Y_s = -\frac12 \alpha^2 (t-s) + \alpha Z\sqrt{t-s} \,,
$$
with $Z\sim\mathcal{N}(0,1).$ Then `base[i,j,k]` is given by
$$
\Delta_j\hat{Y}^{i,k} := \hat{Y}^{i,k}(t^1_{j})-\hat{Y}^{i,k}(t^1_{j-1}) = -\frac12 \alpha_k^2 \textrm{intervals}_{jk} + \alpha_k \cdot \textrm{draws}_{ijk} \cdot \sqrt{\textrm{intervals}_{jk}}\,,
$$
where 
$$
\textrm{intervals}[j,k]= \begin{cases}\textrm{intervals}[j,k]=\text{DCC}(t^1_{j-1}, t^1_{j})\text{ for }j>0 \text{ (independent of } k\text{)}\,,\\
\textrm{intervals}[0,k]=\text{DCC}(t^0_{k}, t^1_{0})\text{ for }j=0\,.
\end{cases}
$$
$\text{DCC}$ represents the day count convention, see [Brigo & Mercurio, 2006], and will depend on the calendar used. For simplicity, we can think of it as $\text{DCC}(t,t')=t'-t$ as a year fraction. Also, 

- `draws[i,j,k]` is a sample of a random variable for the $i$-th simulation and the $k$-th valuation date for the $j$-th step (simulation date).

Thus, a telescopic cancellation gives, by definition using `np.add.accumulate`:
$$
\textrm{paths}[i,j,k]= \begin{cases}\textrm{paths}[i,j,k]=\sum_{j'=0}^{j-1}\Delta_{j'}\hat{Y}^{i,k} = \hat{Y}^{i,k}(t^1_{j-1})-\hat{Y}^{i,k}(t^1_0) = \hat{Y}^{i,k}(t^1_{j-1})-\log(\sigma_0^k)
&\text{ for }j>0 \,,\\
\textrm{paths}[i,0,k]=0&\text{ for }j=0\,.
\end{cases}
$$
Finally, renaming ``paths``:
$$
\hat{\sigma}_{i,k}(t^1_{j-1})=\textrm{paths}[i,j,k]= \begin{cases}\textrm{paths}[i,j,k]=\sigma_0^k\cdot \exp\left(\hat{Y}^{i,k}(t^1_{j-1})-\log(\sigma_0^k)\right) = \exp\left(\hat{Y}^{i,k}(t^1_{j-1})\right)
&\text{ for }j>0 \,,\\
\textrm{paths}[i,0,k]=\sigma_0^k&\text{ for }j=0\,.
\end{cases}
$$


 $\text{Code}$ - math $(\TeX)$ **translation**

* `start_dates`      $\rightarrow \{t^0_k\}_{k=0}^K$ ,
* `draws`    $\rightarrow$     normal draws as above,
* `intervals` $\rightarrow$ intervals as above.

## generate_paths

It returns the simulation paths for the asset price. Following the notation of [Chen et al., 2011], for the $i$-th simulation and the $k$-th valuation date if we denote by `paths` the output of this method we have
$$
\textrm{paths}[i,j,k,l=0] = \hat{S}_{i,k}(t^1_{j-1,k}), \quad j\in\{0, \ldots, J+1\},
$$ 
where 
$$
t^1_{j-1,k}:= \begin{cases} t^1_{j} \qquad \text{ for }j\geq 0 \quad \text{ (independent of } k\text{)}\,,\\
t^0_{k} \qquad \text{ for }j=-1\,,
\end{cases}
$$
which is consistent with the definition of `intervals` given above. The simulation depends on the starting date as, for instance, 
$$
\hat{S}_{i,k}(t^1_{0-1,k}) = \hat{S}_{i,k}(t^0_{k}) = S(t_k^0)\,,
$$
which is a common value for all simulations for that valuation date, the initial condition that would be obtained using market data. Similarly, parameters can depend on the simulation date.

In this case, we have:

- `draws[i,j,k,0]` is a sample of a normal random variable ($W_{2,1}$, p.18) for the $i$-th simulation and the $k$-th valuation date for the $j$-th step (simulation date).
- `draws[i,j,k,1]` is a sample of a normal random variable ($W_{1}=N^{-1}(U_1)$, p.19) for the $i$-th simulation and the $k$-th valuation date for the $j$-th step (simulation date).
- `draws[i,j,k,2]` is a sample of a normal random variable ($Z$ of (3.9)) for the $i$-th simulation and the $k$-th valuation date for the $j$-th step (simulation date).
- `draws[i,j,k,3]` is a sample of a *uniform* random variable ($U$ of p.13) for the $i$-th simulation and the $k$-th valuation date for the $j$-th step (simulation date).




 $\text{Code}$ - math $(\TeX)$ **translation**

* `start_dates`      $\rightarrow \{t^0_k\}_{k=0}^K$ ,
* `step_dates`      $\rightarrow \{t^1_j\}_{j=0}^J$ ,
* `draws`    $\rightarrow$     normal draws as above,
* `intervals` $\rightarrow$ intervals as above.

# References

* [Andersen, 2008] Andersen, L. (2008). Simple and efficient simulation of the Heston stochastic volatility model.
            Journal of Computational Finance, 11(3), 1-42.

* [Brigo & Mercurio, 2006] Brigo, D., & Mercurio, F. (2006). Interest Rate Models: Theory and Practice(Chapter 1). Springer.

* [Chen et al., 2011] Chen, B., Grzelak, L. A., & Oosterlee, C. W. (2011). Calibration and Monte Carlo pricing of the SABR-Hull-White model for long-maturity equity derivatives.
The Journal of Computational Finance (79–113) Volume, 15.

* [Andersen & Piterbarg, 2010] Andersen, L. and Piterbarg, V., (2010). Interest Rate Modeling.
